## Phase 1: Data Understanding & Preparation
## Step 1: Dataset Exploration

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Set paths
BASE_PATH = "src/predictive_analytics/data/complete_set"
TRAIN_PATH = os.path.join(BASE_PATH, "training_set")
TEST_PATH = os.path.join(BASE_PATH, "testing_set")

# Explore dataset structure
def explore_dataset():
    benign_path = os.path.join(TRAIN_PATH, "benign")
    malignant_path = os.path.join(TRAIN_PATH, "malignant")
    
    benign_count = len(os.listdir(benign_path))
    malignant_count = len(os.listdir(malignant_path))
    
    print(f"Training Images:")
    print(f"Benign: {benign_count}")
    print(f"Malignant: {malignant_count}")
    print(f"Total: {benign_count + malignant_count}")
    print(f"Class Balance: {benign_count/(benign_count + malignant_count):.2%} benign")
    
    # Check image properties
    sample_image = cv2.imread(os.path.join(benign_path, os.listdir(benign_path)[0]))
    print(f"\nSample image shape: {sample_image.shape}")
    print(f"Image dtype: {sample_image.dtype}")
    
    return benign_count, malignant_count

benign_count, malignant_count = explore_dataset()